# Progetto Foundations of Computer Science
Membri del gruppo: Andrea Pianalto, Michele Sartori, Silvia Gloria Tamburini

## Descrizione dei database necessari per il progetto

*Kiva* è un sito che fa da tramite tra delle persone che hanno bisogno di denaro (che vivono tipicamente in paesi del secondo o del terzo mondo e che vogliono portare avanti dei progetti), e persone che prestano loro dei soldi. Andando sul loro sito, è possibile infatti essere quello che presta i soldi, oppure essere quello che li riceve: per essere il primo, cliccando su *lend*, si sceglie l'attività a cui prestare e quanti soldi dare; per essere il secondo, cliccando su *borrow*, bisogna far partire una campagna di finanziamento - di fatto, un prestito, un *loan* - per ricevere dei soldi.

Nel seguito, le persone che prestano saranno chiamate *lenders*, mentre quelle che ricevono il prestito sono le *borrowers*. Il singolo prestito è il *loan*, ed è identificato dalla sua chiave: *loan_id*.

**country_stats:** contiene alcune informazioni sul paese da cui provengono i *borrowers*.\
**lenders:** identikit dei *lenders* (lavoro, paese da cui provengono, da quanto tempo prestano, e altro).\
**loans**: informazioni sui singoli prestiti - e quindi, sui *borrowers*. \
**loans_lenders**: associa ogni prestito ai suoi prestatori.

## Importazione delle librerie necessarie (per eseguire tutto il file)

In [165]:
import numpy as np
import pandas as pd
import re
import random

## Importare qui i file quando necessario
Seguire le indicazioni riportate per ogni esercizio e importare solo i file necessari (per non riempire la memoria)

In [2]:
loan_lenders = pd.read_csv('Data/loans_lenders.csv')
loan_lenders['lenders'] = loan_lenders['lenders'].str.split(',\s')

In [3]:
loans = pd.read_csv('Data/loans.csv')
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])

In [66]:
lenders = pd.read_csv('Data/lenders.csv')

In [99]:
countries = pd.read_csv('Data/country_stats.csv')

In [5]:
#Questo file si trova nella cartella dei dati se è stato precedentemente eseguito in qualche momento l'esercizio 1.
norm = pd.read_csv('Data/norm_loan_lenders.csv')

## Esercizio 1
*Consegna*: Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.\
*Traduzione*: Normalizzare la tabella *loan_lenders*, tale che nella tabella normalizzata, ogni riga abbia un *loan_id* e un solo prestatore.

**File necessari**: loan_lenders \
**Avvertenze**: questo esercizio ha un tempo di esecuzione più lungo degli altri (circa 10 minuti)

### Prima soluzione: più veloce ma meno leggibile

In [6]:
norm = pd.DataFrame(columns=['loan_id','lenders'])  
norm = norm.append([dic for i in [[{'loan_id':loan_lenders['loan_id'][row],'lenders':l} for l in loan_lenders['lenders'][row]] for row in range(len(loan_lenders))] for dic in i],ignore_index = True)

### Seconda soluzione: meno veloce ma più leggibile

In [86]:
def dataframe_from_row (index):
    id_loan = loan_lenders.at[index,'loan_id']
    lenders = loan_lenders.at[index,'lenders']
    rep_id = [id_loan]*len(lenders)
    data = {'loan_id':rep_id,'lender_norm':lenders}
    return data

In [87]:
indexes = len(list(loan_lenders.index))

In [99]:
norm = pd.DataFrame(columns = ['loan_id','lender_norm'])
norm = norm.append([pd.DataFrame.from_dict(dataframe_from_row(index)) for index in range(indexes)],ignore_index = True)

Wall time: 2.94 s


### Esportazione del file

In [7]:
norm.to_csv('Data/norm_loan_lenders.csv',index=False)

### Reimportazione del file
Questo svuoterà un po' la memoria dal precedente calcolo, sostituendo *norm* con il file importato.

In [ ]:
norm = pd.read_csv('Data/norm_loan_lenders.csv')

# Esercizio 2
*Consegna*: For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.\
*Traduzione*: per ogni prestito, aggiungere una colonna 'durata' corrispondente al numero di giorni tra il *disburse_time* e il *planned_expiration_time*. Se una di queste due date è mancante, anche la durata deve essere mancante.

**File necessari**: loans

### Visualizzazione del problema

*Disburse time*: indica il momento in cui il *borrower* riceve i suoi fondi. Qui è importante solo la data. \
*Planned expiration time*: momento in cui il prestito scade e bisogna restituire i soldi. Anche qui è importante solo la data.

*Loan length / repayment term*: il numero di mesi che passano dal momento in cui il prestito è dato effettivamente al *borrower* fino a quando bisogna rifondare l'ultimo prestatore.

In [ ]:
loans[['loan_id', 'planned_expiration_time', 'disburse_time']]

### Calcolo della differenza delle due date

In [3]:
date1 = loans['planned_expiration_time'].dt.date
date2 = loans['disburse_time'].dt.date
loans['loan_length'] = date1-date2

In [5]:
loans[['loan_id', 'loan_length']]

,loan_id,loan_length
0,657307,54 days
1,657259,96 days
2,658010,37 days
3,659347,35 days
4,656933,58 days
...,...,...
1419602,988180,40 days
1419603,988213,39 days
1419604,989109,51 days
1419605,989143,63 days


In [95]:
loans['loan_length'].dt.days

0          54.0
1          96.0
2          37.0
3          35.0
4          58.0
           ... 
1419602    40.0
1419603    39.0
1419604    51.0
1419605    63.0
1419606    61.0
Name: loan_length, Length: 1419607, dtype: float64

# Esercizio 3
*Consegna*: Find the lenders that have funded at least twice.\
*Traduzione*: trova i prestatori che hanno dato almeno due prestiti.

**File necessari**: norm

In [117]:
lenders_twice = norm.groupby('lenders').count() > 1
list(lenders_twice[lenders_twice['loan_id'] == True].index)

['000',
 '00000',
 '0002',
 '0101craign0101',
 '0132575',
 '0154884',
 '0161130',
 '0169713',
 '0185429',
 '0197462',
 '0206338',
 '0219854',
 '0221581',
 '0239059',
 '0245597',
 '0256321',
 '0265562',
 '0279282',
 '0288537',
 '0295920',
 '0307987',
 '0321212',
 '0326lsw',
 '0332772',
 '0346439',
 '0353400',
 '0367630',
 '0376099',
 '0384195',
 '0393784',
 '0407067',
 '0416503',
 '0422888',
 '0432352',
 '0443760',
 '0457584',
 '0462602',
 '0473787',
 '0483421',
 '0499990',
 '0509115',
 '0511209',
 '0526528',
 '0545998',
 '0554687',
 '0561575',
 '0579150',
 '0589889',
 '0595846',
 '0609725',
 '0614925',
 '0626305',
 '0634944',
 '0648612',
 '0653266',
 '0672816',
 '0684667',
 '0693181',
 '0703092',
 '070707Weddingtablegifts',
 '0711782',
 '0723706',
 '07272010',
 '0739360',
 '0743222',
 '0755154',
 '0764579',
 '0779467',
 '0786145',
 '0797268',
 '07brit08',
 '0802769',
 '0816',
 '0819212',
 '0822911',
 '0844736',
 '0854755',
 '0858539',
 '0868635',
 '0878881',
 '0894610',
 '0902841',
 '0

# Esercizio 4
*Consegna*: For each country, compute how many loans have involved that country as borrowers.\
*Traduzione*: per ogni paese, calcola quanti prestiti hanno coinvolto quel paese come prestatore.

**File necessari**: loans

In [118]:
loans_per_country = loans.groupby('country_name').count()['loan_id']
loans_per_country

country_name
Afghanistan        2337
Albania            3075
Armenia           13952
Azerbaijan        10172
Belize              218
                  ...  
Vietnam           21839
Virgin Islands        2
Yemen              4206
Zambia             1277
Zimbabwe           5513
Name: loan_id, Length: 96, dtype: int64

# Esercizio 5
*Consegna*: For each country, compute the overall amount of money borrowed.\
*Traduzione*: Per ogni paese, calcola l'ammontare totale di denaro prestato.

**File necessari**: loans

### Visualizzazione del problema

Nel database *loans* ci sono due elementi che sembrano simili: *loan_amount* e *funded amount*. Grazie alle seguenti linee di codice (cambiando il confronto), si nota che generalmente i due sono uguali (1.355.316 volte), ma piuttosto spesso capita anche che *funded_amount* < *loan_amount* (64.279 volte) e qualche volta (12) è il contrario.

Il motivo è questo:\
*loan_amount* è a quanto ammontava la richiesta di prestito.\
*funded _amount* sono i soldi che sono effettivamente arrivati al richiedente.

In [119]:
loans[['loan_id', 'funded_amount', 'loan_amount']]
difference = loans['funded_amount']-loans['loan_amount']
difference[difference==0] #Cambiare questa riga per confrontare i due elementi

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
1419602    0.0
1419603    0.0
1419604    0.0
1419605    0.0
1419606    0.0
Length: 1355316, dtype: float64

### Risoluzione del problema

E' stato usato *loan_amount* perché in una domanda di una discussione era stato confermato dal professore di utilizzare questa cifra. Tuttavia, forse *funded_amount* era più rappresentativo per la quantità richiesta?

In [7]:
money_per_country = loans.groupby('country_name').sum()['loan_amount']
money_per_country

country_name
Afghanistan        1967950.0
Albania            4307350.0
Armenia           22950475.0
Azerbaijan        14784625.0
Belize              150175.0
                     ...    
Vietnam           24681100.0
Virgin Islands       10000.0
Yemen              3444000.0
Zambia             1978975.0
Zimbabwe           5851875.0
Name: loan_amount, Length: 96, dtype: float64

# Esercizio 6
*Consegna*: Like the previous point, but expressed as a percentage of the overall amount lent. \
*Traduzione*: come il punto precedente (cioè: per ogni paese, calcola quanti soldi ha prestato), ma espresso in percentuale rispetto al totale prestato.

**File necessari**: nessuno, ma deve essere definito _money_per_country_ (esercizio 5)

In [121]:
money_per_country_percentage = money_per_country/money_per_country.sum()*100
money_per_country_percentage

country_name
Afghanistan       0.166573
Albania           0.364586
Armenia           1.942589
Azerbaijan        1.251410
Belize            0.012711
                    ...   
Vietnam           2.089074
Virgin Islands    0.000846
Yemen             0.291509
Zambia            0.167506
Zimbabwe          0.495318
Name: loan_amount, Length: 96, dtype: float64

# Esercizio 7
*Consegna*: Like the three previous points, but split for each year (with respect to *disburse time*).\
*Traduzione*: come nei tre punti precedenti (esercizi 4-5-6), ma separatamente per ogni anno, rispetto a *disburse_time*.

**File necessari**: loans

### Aggiunta dell'anno rispetto al quale dividere i risultati

In [122]:
loans['disburse_year'] = loans['disburse_time'].dt.year
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,loan_length,disburse_year
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,3,2,1,NaN,female,true,irregular,field_partner,54 days,2013.0
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,11,2,1,NaN,female,true,monthly,field_partner,96 days,2013.0
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days,2014.0
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,21,2,1,user_favorite,female,true,monthly,field_partner,35 days,2014.0
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,58 days,2013.0


### Analogo del punto 4.
*Consegna*: For each country, compute how many loans have involved that country as borrowers\
*Traduzione*: per ogni paese, calcola quanti prestiti hanno coinvolto quel paese come prestatore.

In [123]:
loans_per_country_per_year = loans.groupby(['country_name','disburse_year']).count()['loan_id']
loans_per_country_per_year

country_name  disburse_year
Afghanistan   2007.0            408
              2008.0            370
              2009.0            678
              2010.0            632
              2011.0            247
                               ... 
Zimbabwe      2013.0            426
              2014.0           2078
              2015.0            600
              2016.0            808
              2017.0           1079
Name: loan_id, Length: 748, dtype: int64

### Analogo del punto 5
*Consegna*: For each country, compute the overall amount of money borrowed.\
*Traduzione*: per ogni paese, calcola l'ammontare totale del denaro prestato.

In [125]:
money_per_country_per_year = loans.groupby(['country_name','disburse_year'])['loan_amount'].sum()
money_per_country_per_year

country_name  disburse_year
Afghanistan   2007.0            194975.0
              2008.0            365375.0
              2009.0            585125.0
              2010.0            563350.0
              2011.0            245125.0
                                 ...    
Zimbabwe      2013.0            678525.0
              2014.0           1311575.0
              2015.0            723625.0
              2016.0            788600.0
              2017.0           1237600.0
Name: loan_amount, Length: 748, dtype: float64

### Analogo del punto 6
*Consegna*: Like the previous point, but expressed as a percentage of the overall amount lent.\
*Traduzione*: come nel punto precedente (calcola l'ammontare totale del denaro prestato per ogni paese), ma espresso in percentuale rispetto al totale.

In [126]:
money_per_country_percentage_per_year = money_per_country_per_year/money_per_country_per_year.sum()*100
money_per_country_percentage_per_year

country_name  disburse_year
Afghanistan   2007.0           0.016657
              2008.0           0.031215
              2009.0           0.049989
              2010.0           0.048129
              2011.0           0.020942
                                 ...   
Zimbabwe      2013.0           0.057969
              2014.0           0.112053
              2015.0           0.061822
              2016.0           0.067373
              2017.0           0.105733
Name: loan_amount, Length: 748, dtype: float64

# Esercizio 8
*Consegna*: For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.\
*Traduzione*: per ogni prestatore, calcola la somma totale di denaro che ha prestato. Per ogni prestito che include più di un prestatore, devi assumere che tutti i prestatori abbiano contribuito in modo uguale.

**File necessari**: loan_lenders, norm e loans.\
**Avvertenze**: questo esercizio può riempire la memoria (75% dei miei 16GB). E' consigliabile eseguirlo separatamente dagli altri, ed eventualmente, se necessario, eseguire l'esercizio 9 in successione. 

### Procedimento
Si parte da *loan_lenders*, e per ogni prestito si calcola:

* Quanti sono i contribuenti (da *norm*)
* L'ammontare del prestito (da *loans*)
* La spesa per ogni contribuente.
* Chi sono i contribuenti (da *norm*)

Poi si farà un raggruppamento per contribuente.

### Osservazione
Il numero di prestatori per ogni *loan_id* è indicato nella tabella *loans*, e facendo un confronto, si è osservato che tale numero è diverso da quello trovato raggruppando il file *norm_loan_lenders* per *loan_id* e contando gli elementi per gruppo. Tuttavia, poiché in *loans* non sono indicati i nomi dei prestatori, è impossibile determinare chi ha prestato a chi; per questa ragione, si è deciso di fare riferimento al raggruppamento di *norm_loan_lenders*.

### a. Quanti sono i contribuenti

Si determina, per ogni prestito, quanti contribuenti ha

In [6]:
lenders_per_loan = norm.groupby('loan_id').count()
lenders_per_loan

,lenders
loan_id,
84,3
85,2
86,3
88,3
89,4
...,...
1444051,1
1444053,1
1444058,1


Si aggiunge questa informazione a *loan_lenders*

In [7]:
loan_lenders = loan_lenders.merge(lenders_per_loan, on='loan_id')
loan_lenders

,loan_id,lenders_x,lenders_y
0,483693,"[muc888, sam4326, camaran3922, lachheb1865, re...",40
1,483738,"[muc888, nora3555, williammanashi, barbara5610...",15
2,485000,"[muc888, terrystl, richardandsusan8352, sherri...",15
3,486087,"[muc888, james5068, rudi5955, daniel9859, don9...",13
4,534428,"[muc888, niki3008, teresa9174, mike4896, david...",19
...,...,...,...
1387427,678999,"[michael43411218, carol5987, gooddogg1, chris4...",10
1387428,1207353,"[rjhoward1986, jeffrey6870, trolltech4460, ely...",5
1387429,1206220,"[vicky7746, gooddogg1, fairspirit, craig972996...",44
1387430,1206425,"[rich6705, sergiiy9766, angela7509, barbara561...",8


Si pulisce la tabella rinominando le colonne ed eliminando quelle inutili.

In [8]:
#Pulizia della tabella: rinominare le colonne.
loan_lenders = loan_lenders.rename({'lenders_x': 'list_of_lenders', 'lenders_y': 'number_of_lenders'}, axis=1)
loan_lenders

,loan_id,list_of_lenders,number_of_lenders
0,483693,"[muc888, sam4326, camaran3922, lachheb1865, re...",40
1,483738,"[muc888, nora3555, williammanashi, barbara5610...",15
2,485000,"[muc888, terrystl, richardandsusan8352, sherri...",15
3,486087,"[muc888, james5068, rudi5955, daniel9859, don9...",13
4,534428,"[muc888, niki3008, teresa9174, mike4896, david...",19
...,...,...,...
1387427,678999,"[michael43411218, carol5987, gooddogg1, chris4...",10
1387428,1207353,"[rjhoward1986, jeffrey6870, trolltech4460, ely...",5
1387429,1206220,"[vicky7746, gooddogg1, fairspirit, craig972996...",44
1387430,1206425,"[rich6705, sergiiy9766, angela7509, barbara561...",8


In [9]:
#Pulizia della tabella: eliminare le colonne inutili.
loan_lenders = loan_lenders.drop(labels='list_of_lenders', axis=1)
loan_lenders

,loan_id,number_of_lenders
0,483693,40
1,483738,15
2,485000,15
3,486087,13
4,534428,19
...,...,...
1387427,678999,10
1387428,1207353,5
1387429,1206220,44
1387430,1206425,8


### b. A quanto ammonta il prestito

In [10]:
loan_lenders = loan_lenders.merge(loans[['loan_id', 'loan_amount']], on='loan_id', copy=False)
loan_lenders

,loan_id,number_of_lenders,loan_amount
0,483693,40,1225.0
1,483738,15,500.0
2,485000,15,725.0
3,486087,13,400.0
4,534428,19,625.0
...,...,...,...
1387423,678999,10,325.0
1387424,1207353,5,200.0
1387425,1206220,44,2175.0
1387426,1206425,8,325.0


### c. Calcolo della singola contribuzione

In [11]:
loan_lenders['single_contribution'] = loan_lenders['loan_amount']/loan_lenders['number_of_lenders']
loan_lenders

,loan_id,number_of_lenders,loan_amount,single_contribution
0,483693,40,1225.0,30.625000
1,483738,15,500.0,33.333333
2,485000,15,725.0,48.333333
3,486087,13,400.0,30.769231
4,534428,19,625.0,32.894737
...,...,...,...,...
1387423,678999,10,325.0,32.500000
1387424,1207353,5,200.0,40.000000
1387425,1206220,44,2175.0,49.431818
1387426,1206425,8,325.0,40.625000


### d. Chi sono i contribuenti

In [12]:
norm = norm.merge(loan_lenders[['loan_id', 'single_contribution']], how='left', on='loan_id')
norm.head()

,loan_id,lenders,single_contribution
0,483693,muc888,30.625
1,483693,sam4326,30.625
2,483693,camaran3922,30.625
3,483693,lachheb1865,30.625
4,483693,rebecca3499,30.625


### Risultato finale

In [74]:
norm_with_contributions = norm.groupby('lenders').sum()

In [75]:
norm_with_contributions

,loan_id,single_contribution
lenders,,
000,23403663,1764.285078
00000,6526719,1380.693644
0002,21825027,2472.563566
00mike00,502260,52.631579
0101craign0101,16421751,2623.565117
...,...,...
zzmcfate,1030584076,66113.226325
zzpaghetti9994,1082908,51.020408
zzrvmf8538,11473337,576.978086


# Esercizio 9
*Consegna*: For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.\
*Traduzione*: Per ogni paese, calcola la differenza tra il denaro prestato (spese) e il denaro richiesto (guadagni). Poiché il paese del prestatore è spesso non noto, si può assumere che la vera distribuzione dei paesi sia analoga a quella calcolata a partire dalle righe dove il paese è noto.

**File necessari**: loans, lenders, countries, norm_with_contributions (dall'esercizio 8)\
**Avvertenze**: questo esercizio non riempie la memoria di più dell'esercizio 8, ma valgono comunque le sue avvertenze.

### *country_code* e *country_name*: il caso della Namibia.
Controllando, si è notato che in *loans* ci sono nove casi in cui *country_code* manca, mentre *country_name* non manca mai; questi nove casi riguardano sempre la Namibia. Poiché neanche in *countries* la Namibia aveva un codice, e il codice 'NA' non ce l'aveva nessuno, si è pensato di assegnarlo alla Namibia.

In [17]:
loans[loans['country_code'].isnull()]['country_name']

82889      Namibia
156970     Namibia
598087     Namibia
684876     Namibia
971827     Namibia
1134818    Namibia
1214923    Namibia
1281022    Namibia
1415763    Namibia
Name: country_name, dtype: object

In [18]:
loans[loans['country_name'].isnull()]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model


In [19]:
countries[countries['country_name'] == 'Namibia']
countries[countries['country_code'] == 'NA']

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name


In [20]:
for i in countries[countries['country_name'] == 'Namibia'].index:
    countries.at[i, 'country_code'] = 'NA'
    
for i in loans[loans['country_name'] == 'Namibia'].index:
    loans.at[i, 'country_code'] = 'NA'

In [21]:
countries[countries['country_code'].isnull()]['country_name']

Series([], Name: country_name, dtype: object)

### Calcolo delle entrate per ogni paese (soldi ricevuti da prestiti)

In [50]:
earnings_by_country_name = loans.groupby('country_name').sum()
earnings_by_country_name

,loan_id,funded_amount,loan_amount,currency_exchange_coverage_rate,partner_id,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries
country_name,,,,,,,,,
Afghanistan,310988745,1964650.0,1967950.0,0.2,80214.0,31641.0,60171,5563,2807
Albania,2704828308,3992475.0,4307350.0,307.5,735153.0,71863.0,133105,4857,3560
Armenia,11915451816,20284675.0,22950475.0,1277.7,2218759.0,385090.0,598715,29153,19713
Azerbaijan,3369911168,14191350.0,14784625.0,283.5,528703.0,166800.0,416868,15576,13069
Belize,156667303,150175.0,150175.0,21.8,59514.0,7646.0,5523,574,354
...,...,...,...,...,...,...,...,...,...
Vietnam,16461180213,23463825.0,24681100.0,1876.6,3609109.0,343021.0,706699,29436,22767
Virgin Islands,2163307,0.0,10000.0,0.0,0.0,48.0,0,2,2
Yemen,2964702949,3340025.0,3444000.0,436.9,862230.0,67082.0,96519,7703,5710


### Calcolo delle uscite per ogni paese (soldi dati come prestiti)

I soldi dati in prestito vanno controllati da *lenders*, in cui non c'è *country_name*, ma solo *country_code*. Inoltre, c'è il problema che in *lenders*, *country_code* è quasi sempre vuoto.

#### Per ogni prestatore, unione delle sue contribuzioni e del suo paese. 

In [51]:
expenses_by_country_name = pd.merge(lenders[['permanent_name', 'country_code']], norm_with_contributions, left_on = 'permanent_name',right_on = 'lenders')
expenses_by_country_name.head()

,permanent_name,country_code,loan_id,single_contribution
0,qian3013,NaN,1050447,46.839080
1,reena6733,NaN,10858522,417.948366
2,andrew86079135,NaN,5706153,349.601088
3,sirinapa6764,NaN,1055111,50.000000
4,rene7585,NaN,2112248,59.126984


#### Calcolo della frequenza dei paesi (normalizzata a 1)

In [52]:
#Frequenza assoluta dei codici dei paesi.
codes_per_country = expenses_by_country_name.dropna().groupby('country_code').count()['permanent_name']

#Numero dei codici disponibili
existing_codes = len(expenses_by_country_name.dropna())

#Frequenza relativa dei codici dei paesi.
frequency = codes_per_country/existing_codes
frequency

country_code
AD    0.000014
AE    0.001019
AF    0.000189
AG    0.000005
AI    0.000002
        ...   
YE    0.000078
YT    0.000002
ZA    0.001007
ZM    0.000026
ZW    0.000018
Name: permanent_name, Length: 225, dtype: float64

#### Estrazione dei paesi che mancano (per riempire i nulli)

In [53]:
extract_set = list(frequency.index) #Set da cui estrarre i valori
extract_prob = list(frequency.values) #Probabilità con cui estrarli
extract_num = len(expenses_by_country_name[expenses_by_country_name['country_code'].isnull()]) #Valori da estrarre

new_countries = list(np.random.choice(extract_set, extract_num, True, extract_prob))
new_countries

['US',
 'US',
 'SE',
 'US',
 'US',
 'GB',
 'CO',
 'US',
 'US',
 'AU',
 'US',
 'US',
 'US',
 'US',
 'DE',
 'GB',
 'US',
 'US',
 'US',
 'CA',
 'US',
 'US',
 'US',
 'US',
 'AU',
 'US',
 'US',
 'US',
 'PT',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'DE',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'IE',
 'US',
 'CA',
 'NZ',
 'US',
 'US',
 'US',
 'GB',
 'US',
 'US',
 'US',
 'US',
 'US',
 'CA',
 'US',
 'US',
 'US',
 'US',
 'US',
 'AU',
 'US',
 'NO',
 'US',
 'US',
 'GB',
 'CA',
 'US',
 'US',
 'GB',
 'US',
 'US',
 'CH',
 'US',
 'CA',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'GB',
 'CA',
 'CA',
 'CA',
 'US',
 'CA',
 'US',
 'US',
 'US',
 'AU',
 'US',
 'NL',
 'US',
 'US',
 'DE',
 'AU',
 'AU',
 'PH',
 'US',
 'CA',
 'US',
 'US',
 'US',
 'US',
 'US',
 'AU',
 'US',
 'NZ',
 'AU',
 'US',
 'US',
 'ES',
 'US',
 'US',
 'US',
 'CA',
 'US',
 'US',
 'US',
 'AU',
 'US',
 'US',
 'US',
 'US',
 'CZ',
 'US',
 'CA',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'CA',
 'US',
 'US',
 'US',
 'US',

#### Riempimento dei valori nulli dei paesi.

In [54]:
keys = list(expenses_by_country_name[expenses_by_country_name['country_code'].isnull()].index)

#Riempi i nulli della serie country_code con 
expenses_by_country_name['country_code'] = expenses_by_country_name['country_code'].fillna(dict(zip(keys,new_countries)))

In [55]:
expenses_by_country_name[expenses_by_country_name['country_code'].isnull()]

,permanent_name,country_code,loan_id,single_contribution


#### Lenders e *country_code*
Siccome *lenders* non ha il *country_name* ma solo il *country_code*, si è pensato di assegnarglielo tramite un merge a partire da *countries*.

In [56]:
expenses_by_country_name = expenses_by_country_name.merge(countries[['country_code', 'country_name']], how='left', on='country_code')
expenses_by_country_name.head()

,permanent_name,country_code,loan_id,single_contribution,country_name
0,qian3013,US,1050447,46.839080,United States
1,reena6733,US,10858522,417.948366,United States
2,andrew86079135,SE,5706153,349.601088,Sweden
3,sirinapa6764,US,1055111,50.000000,United States
4,rene7585,US,2112248,59.126984,United States


#### Calcolo effettivo delle spese sostenute da un paese

In [57]:
#Funded amount è quello di norm_with_contributions
expenses_by_country_name = expenses_by_country_name.groupby('country_name').sum()
expenses_by_country_name

,loan_id,single_contribution
country_name,,
Afghanistan,1995654545,134388.869411
Albania,228373636,12805.782967
Algeria,341841359,16810.573874
Angola,660507832,38329.010744
Argentina,8666979823,456238.905013
...,...,...
Venezuela,1733321927,109167.387720
VietNam,6913699469,363370.655956
Yemen,604364386,37722.187724


### Risultato: bilancio complessivo

#### Calcolo del bilancio

In [101]:
report_by_country_name = expenses_by_country_name.merge(earnings_by_country_name[['loan_id', 'loan_amount']], on='country_name', how='outer')
report_by_country_name.head()

,loan_id_x,single_contribution,loan_id_y,loan_amount
country_name,,,,
Afghanistan,1.995655e+09,134388.869411,3.109887e+08,1967950.0
Albania,2.283736e+08,12805.782967,2.704828e+09,4307350.0
Algeria,3.418414e+08,16810.573874,NaN,NaN
Angola,6.605078e+08,38329.010744,NaN,NaN
Argentina,8.666980e+09,456238.905013,NaN,NaN


#### Pulizia del dataset del bilancio

In [102]:
report_by_country_name = report_by_country_name.drop(labels=['loan_id_x', 'loan_id_y'], axis=1)
report_by_country_name = report_by_country_name.rename({'single_contribution': 'expenses', 'loan_amount': 'earnings'}, axis=1)
report_by_country_name.head()

,expenses,earnings
country_name,,
Afghanistan,134388.869411,1967950.0
Albania,12805.782967,4307350.0
Algeria,16810.573874,NaN
Angola,38329.010744,NaN
Argentina,456238.905013,NaN


#### Riempimento dei valori nulli
Ci sono dei paesi che hanno solo *lenders*, e paesi che hanno solo *borrowers*, e quindi ci sono dei valori nulli, che riempiamo con zero.

In [103]:
report_by_country_name = report_by_country_name.fillna(0.0)

#### Colonna con il bilancio complessivo

In [104]:
#Denaro prestato - denaro richiesto.
report_by_country_name['total'] = report_by_country_name['expenses'] - report_by_country_name['earnings']
report_by_country_name['total']

country_name
Afghanistan                            -1.833561e+06
Albania                                -4.294544e+06
Algeria                                 1.681057e+04
Angola                                  3.832901e+04
Argentina                               4.562389e+05
                                            ...     
Saint Vincent and the Grenadines       -4.922500e+04
The Democratic Republic of the Congo   -1.881342e+07
Vanuatu                                -9.250000e+03
Vietnam                                -2.468110e+07
Virgin Islands                         -1.000000e+04
Name: total, Length: 180, dtype: float64

# Esercizio 10
*Consegna*: Which country has the highest ratio between the difference computed at the previous point and the population?\
*Traduzione*: Quale paese ha il rapporto massimo tra la differenza calcolata al punto precedente e la popolazione?

**File necessari**: report_by_country_name (dall'esercizio 9), e in più countries (che già serve per l'esercizio 9)

### Aggiunta delle informazioni sulla popolazione

In [105]:
report_by_country_name = report_by_country_name.merge(countries[['country_name','population','population_below_poverty_line']], on='country_name', left_index = True)
report_by_country_name

,country_name,expenses,earnings,total,population,population_below_poverty_line
28,Afghanistan,134388.869411,1967950.0,-1.833561e+06,35530081,35.8
131,Albania,12805.782967,4307350.0,-4.294544e+06,2930187,14.3
39,Algeria,16810.573874,0.0,1.681057e+04,41318142,23.0
29,Angola,38329.010744,0.0,3.832901e+04,29784193,40.5
22,Argentina,456238.905013,0.0,4.562389e+05,44271041,32.2
...,...,...,...,...,...,...
34,Zambia,36410.188894,1978975.0,-1.942565e+06,17094130,60.5
30,Zimbabwe,7660.707232,5851875.0,-5.844214e+06,16529904,72.3
53,Chad,0.000000,20075.0,-2.007500e+04,14899994,46.7
101,Mauritania,0.000000,15000.0,-1.500000e+04,4420184,31.0


### Calcolo del rapporto richiesto

In [106]:
report_by_country_name['ratio_pop_report'] = report_by_country_name['total']/report_by_country_name['population']
report_by_country_name.head()

,country_name,expenses,earnings,total,population,population_below_poverty_line,ratio_pop_report
28,Afghanistan,134388.869411,1967950.0,-1.833561e+06,35530081,35.8,-0.051606
131,Albania,12805.782967,4307350.0,-4.294544e+06,2930187,14.3,-1.465621
39,Algeria,16810.573874,0.0,1.681057e+04,41318142,23.0,0.000407
29,Angola,38329.010744,0.0,3.832901e+04,29784193,40.5,0.001287
22,Argentina,456238.905013,0.0,4.562389e+05,44271041,32.2,0.010306


### Individuazione del massimo

In [107]:
report_by_country_name.loc[report_by_country_name['ratio_pop_report'].idxmax()]

country_name                          Norway
expenses                         2.09171e+07
earnings                                   0
total                            2.09171e+07
population                           5305383
population_below_poverty_line            NaN
ratio_pop_report                     3.94261
Name: 162, dtype: object

# Esercizio 11
*Consegna*: Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?\
*Traduzione*: quale paese ha il rapporto maggiore tra la differenza calcolata al punto 9 e la popolazione che non è sotto alla linea della povertà?

**File necessari**: report_by_country_name (per come è stato modificato dall'esercizio 10)

### Il problema dei valori nulli.
Ci sono dei valori per cui non è nota la percentuale della popolazione sotto la linea della povertà. Visto che non è possibile riempirli da altri elementi del dataset, e non è possibile neanche porli tutti a un unico valore (la Nord Corea, Cuba e la Norvegia ci aspettiamo abbiano valori di popolazione sotto alla linea di povertà molto diversi tra loro) o estrarli a caso come fatto in precedenza (perché questo valore ha una grande rilevanza dal punto di vista politico per un paese). Scegliamo quindi di lasciarli come valori nulli. Si tenga quindi conto che il valore del massimo non terrà conto di questi valori.

In [108]:
report_by_country_name[report_by_country_name['population_below_poverty_line'].isnull()]

,country_name,expenses,earnings,total,population,population_below_poverty_line,ratio_pop_report
151,Australia,6.133676e+07,0.0,6.133676e+07,24450561,NaN,2.508604
152,Bahrain,8.955650e+04,0.0,8.955650e+04,1492584,NaN,0.060001
153,Brunei,3.260978e+04,0.0,3.260978e+04,428697,NaN,0.076067
154,Cuba,3.897320e+03,0.0,3.897320e+03,11484636,NaN,0.000339
155,Cyprus,1.194751e+05,0.0,1.194751e+05,1179551,NaN,0.101289
156,Finland,6.668237e+06,0.0,6.668237e+06,5523231,NaN,1.207307
157,Iceland,8.626198e+05,0.0,8.626198e+05,335025,NaN,2.574792
158,Kuwait,6.873648e+05,0.0,6.873648e+05,4136528,NaN,0.166170
159,Luxembourg,8.863383e+05,0.0,8.863383e+05,583455,NaN,1.519120
160,New Zealand,5.404327e+06,0.0,5.404327e+06,4705818,NaN,1.148435


### Calcolo della popolazione che non è sotto alla linea di povertà

In [111]:
report_by_country_name['population_not_poor'] = report_by_country_name['population'] - \
(report_by_country_name['population_below_poverty_line']/100) * report_by_country_name['population']
report_by_country_name.head()

,country_name,expenses,earnings,total,population,population_below_poverty_line,ratio_pop_report,population_not_poor
28,Afghanistan,134388.869411,1967950.0,-1.833561e+06,35530081,35.8,-0.051606,2.281031e+07
131,Albania,12805.782967,4307350.0,-4.294544e+06,2930187,14.3,-1.465621,2.511170e+06
39,Algeria,16810.573874,0.0,1.681057e+04,41318142,23.0,0.000407,3.181497e+07
29,Angola,38329.010744,0.0,3.832901e+04,29784193,40.5,0.001287,1.772159e+07
22,Argentina,456238.905013,0.0,4.562389e+05,44271041,32.2,0.010306,3.001577e+07


### Calcolo del nuovo rapporto richiesto

In [112]:
report_by_country_name['ratio_pop_not_poor_report'] = report_by_country_name['total']/report_by_country_name['population_not_poor']
report_by_country_name.head()

,country_name,expenses,earnings,total,population,population_below_poverty_line,ratio_pop_report,population_not_poor,ratio_pop_not_poor_report
28,Afghanistan,134388.869411,1967950.0,-1.833561e+06,35530081,35.8,-0.051606,2.281031e+07,-0.080383
131,Albania,12805.782967,4307350.0,-4.294544e+06,2930187,14.3,-1.465621,2.511170e+06,-1.710176
39,Algeria,16810.573874,0.0,1.681057e+04,41318142,23.0,0.000407,3.181497e+07,0.000528
29,Angola,38329.010744,0.0,3.832901e+04,29784193,40.5,0.001287,1.772159e+07,0.002163
22,Argentina,456238.905013,0.0,4.562389e+05,44271041,32.2,0.010306,3.001577e+07,0.015200


### Individuazione del massimo

In [115]:
report_by_country_name.loc[report_by_country_name['ratio_pop_not_poor_report'].idxmax()]

country_name                          Canada
expenses                         1.03043e+08
earnings                               50000
total                            1.02993e+08
population                          36624199
population_below_poverty_line            9.4
ratio_pop_report                     2.81216
population_not_poor              3.31815e+07
ratio_pop_not_poor_report            3.10393
Name: 72, dtype: object

# Esercizio 12
*Consegna*: For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

*Traduzione*: per ogni anno, calcola l'ammontare totale di denaro prestato. Ogni prestito che ha date di scadenza e date di sborso in anni diversi deve avere il suo ammontare distribuito in modo proporziale al numero di giorni in ogni anno. Per esempio, se un prestito ha come data di esborso l'1 Dicembre 2016 e come data di scadenza il 30 Gennaio 2018, ed era di 5000 dollari statunitensi, contribuisce per 363.85 dollari per il 2016, per 4284.04 dollari per il 2017 e per 352.11 per il 2018. 

**File necessari**: loans, modificato come nell'esercizio 2 (con la colonna *Loan_length*)

### Procedimento. 

L'idea è quella di calcolare per ogni prestito la suddivisione delle spese come nell'esempio. 

* Tipo 1: *2014 - 2014*. Entrambe le date sono nello stesso anno. \
$\rightarrow$ In questo caso, la spesa va imputata tutta all'unico anno che compare.

* Tipo 2: *2014-2015*. Le date sono in due anni diversi, ma consecutivi. \
$\rightarrow$ In questo caso, la spesa va imputata in parte al primo anno e in parte al secondo anno, in base a quanti giorni appartengono a quell'anno. 

* Tipo 3: *2014-2016*. Le date sono in due anni diversi, e non consecutivi.\
$\rightarrow$ In questo caso, la spesa va imputata in parte al primo anno, in parte all'ultimo anno (in base a quanti giorni dell'intervallo appartengono a quell'anno), e in parte agli anni consecutivi. 

Di consequenza, per ogni prestito si vuole calcolare innanzitutto la colonna *money_per_day*, contenente il denaro che corrisponde a un giorno. Poi, si sono aggiunte quattro colonne:

* Colonna *single_year_amount*, per tenere conto delle date di tipo 1
* Colonna *first_year_amount* e *last_year_amount*, per tenere conto delle date di tipo 2 e 3.
* Colonna *midterm_year_amount*, per tenere conto delle date di tipo 3.

Infine, per ognuno dei prestiti, in base alla durata del prestito e in base ai giorni degli anni che ricopre, è necessario riempire queste colonne in modo proporzionale. Poi sarà sufficiente raggruppare per gli anni a cui corrispondono e sommare, e si otterrà la quantità richiesta.

### Colonna *money_per_day*.

Contiene la quantità di denaro che, per ogni prestito, corrisponde a ogni giorno di durata del prestito.

In [4]:
loans['money_per_day'] = loans['loan_amount']/loans['loan_length'].dt.days

### Pulizia della colonna *money_per_day* e del database in generale.

* Ci sono dei casi in cui *loan_length.days* vale zero, perché il tempo di prestito è pari a poche ore. In questo caso, *money_per_day* risulta infinito. \
$\rightarrow$ Per semplificare i conti, imponiamo che in questo caso *money_per_day* valga NaN. 

* Ci sono dei NaN anche in altre colonne, come *loan_amount*, *disburse_time* e *planned_expiration_time*.\
$\rightarrow$ Per semplificare i conti, rimuoviamo tutti i valori nulli.

* Ci sono dei casi in cui *loan_length.days* è negativo, perché le due date sono invertite. In questo caso, *money_per_day* risulta negativo.\
$\rightarrow$ Per semplificare i conti, scegliamo di non calcolare tali prestiti.

In [5]:
#Rimozione degli infiniti.
loans = loans.replace([np.inf, -np.inf], np.nan)

#Eliminazione dei valori nulli. 
loans = loans.dropna(subset=['money_per_day', 'disburse_time', 'planned_expiration_time'])

#Rimozione dei valori negativi.
for i in loans[loans['money_per_day'] < 0].index:
    loans.at[i,'loan_amount'] = 0
    loans.at[i,'money_per_day'] = 0

### Creazione delle colonne degli anni e delle spese

Qui si creano le colonne con le spese per anno e le colonne con gli anni, rispetto ai quali poi si dovrà raggruppare.

In [7]:
#Anno inizio e anno di fine
loans['first_year'] = loans['disburse_time'].dt.year.astype(int)
loans['last_year'] = loans['planned_expiration_time'].dt.year.astype(int)
loans['difference_year'] = loans['last_year'] - loans['first_year']

#4 nuove colonne: stesso anno, oppure due anni (inizio e fine), oppure anni in mezzo (se tre o più)
loans['single_year_amount'] = 0.0
loans['first_year_amount'] = 0.0
loans['last_year_amount'] = 0.0
loans['midterm_year_amount'] = 0.0 #Serve perché ci sono 320 casi in cui la differenza tra gli anni è maggiore di 1!

### Riempimento delle colonne a seconda degli anni coinvolti.

* Data di tipo 1: singolo anno.\
$\rightarrow$ *single_year_amount* contiene l'ammontare di tutto il prestito. 

* Data di tipo 2 e 3 - dove c'è un anno di inizio e un anno di fine.\
$\rightarrow$ *first_year_amount* contiene la parte di prestito corrispondente ai giorni dalla data di inizio fino al 1 Gennaio dell'anno dopo (cioè, 365/366 meno il giorno dell'anno della data di inizio). \
$\rightarrow$ *last_year_amount* contiene la parte di prestito corrispondente ai giorni dal 1 Gennaio dell'anno di fine fino alla data di fine (cioè, il giorno dell'anno della data di fine).

* Data di tipo 3: dove ci sono degli anni in mezzo.\
*midterm_year_amount* contiene la parte di prestito corrispondente ai giorni di un anno in mezzo (365 o 366).

NOTA: la somma di tutte queste colonne NON FA SEMPRE il prestito totale. Questo perché nelle date di tipo 3, per come le abbiamo definite, non possiamo tenere conto degli anni bisestili. Questo non è un problema, per il momento: ne teniamo conto dopo.

In [8]:
#Date di tipo 1: singolo anno.
for i in loans[loans['difference_year'] == 0].index:
    loans.at[i, 'single_year_amount'] = loans.at[i, 'loan_amount']

#Date di tipo 2 e 3: dove c'è un anno di inizio e uno di fine.
for i in loans[loans['difference_year'] > 0].index:
    if loans.at[i,'disburse_time'].is_leap_year:
        days_first_year = float(366 - loans.at[i, 'disburse_time'].dayofyear)
    else:
        days_first_year = float(365 - loans.at[i, 'disburse_time'].dayofyear)
    
    days_last_year = float(loans.at[i, 'planned_expiration_time'].dayofyear)
    
    loans.at[i, 'first_year_amount'] = (days_first_year)*loans.at[i, 'money_per_day']
    loans.at[i, 'last_year_amount'] = (days_last_year)*loans.at[i, 'money_per_day']

#Date di tipo 3: dove ci sono degli anni in mezzo. 
for i in loans[loans['difference_year'] > 1].index:
    loans.at[i, 'midterm_year_amount'] = 365.0*loans.at[i, 'money_per_day']

### Riempimento di un dizionario che contiene il risultato.

In [10]:
tot_year = {}

#Tipo 2014-2014 (raggruppo per first year - indifferente - e sommo su single year amount)
for year in list(loans['first_year'].unique()):
    tot_year[year] = tot_year.get(year,0) + loans.groupby('first_year').sum()['single_year_amount'].loc[year]

#Tipo 2014-2015 oppure tipo 2014-2016: anno iniziale
#Raggruppare per first year e sommare su first year amount
for year in list(loans['first_year'].unique()):
    tot_year[year] = tot_year.get(year,0) + loans.groupby('first_year').sum()['first_year_amount'].loc[year]

#Tipo 2014-2015 oppure tipo 2014-2016: anno finale.
#Raggruppare per last year e sommare su last year amount
for year in list(loans['last_year'].unique()):
    tot_year[year] = tot_year.get(year,0) + loans.groupby('last_year').sum()['last_year_amount'].loc[year]

#Tipo 2014-2016: anni in mezzo. 
#Operiamo riga per riga, ma solo per quegli anni. 
for row in loans[loans['difference_year'] > 1].index:
    
    #Quali sono gli anni in mezzo:
    years = list(range(loans.at[row,'first_year']+1,loans.at[row,'last_year']))
    
    for mid_year in years:
        
        tot_year[mid_year] = tot_year.get(mid_year,0) + loans.at[row,'midterm_year_amount']
        
        #Teniamo conto degli anni bisestili: se l'anno è bisestile, aggiungiamo un giorno di pagamento.
        if (mid_year%4 == 0) and (mid_year%400 != 0):
            tot_year[mid_year] += loans.at[row,'money_per_day']

### Arrotondamento del risultato.

In [11]:
for key in tot_year.keys():
    tot_year[key] = round(tot_year[key], 2)
    
tot_year

### Confronto per verificare

In [13]:
loans['loan_amount'].sum()

866530525.0

In [14]:
sum(tot_year.values())

866530524.99

# Esercizio 13

*Consegna*: For each value of repayment_interval, add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state.

*Traduzione*: per ogni valore di *repayment_interval*, aggiunti una nuova colonna al dataframe *lenders*, che contiene l'ammontare totale di denaro corrispondente ai prestiti in quello stato.

**File necessari**: norm, modificato come nell'esercizio 8 (quindi, di conseguenza, tutti i file dell'esercizio 8), e lenders.

### Procedimento.

Il database *loans*, tra le varie informazioni che contiene, contiene anche un'informazione chiamata *repayment_interval*, che sul sito ufficiale di Kiva viene chiamata *repayment_schedule*, che descrive la frequenza con la quale ogni *borrower* restituisce i soldi prestati al *lender*; può essere:

- *weekly*: un ripagamento alla settimana;
- *monthly*: un ripagamento al mese;
- *bullet*: un ripagamento fatto alla fine della durata del prestito;
- *irregular*: ogni altro metodo. 

Per associare a ogni *lender* l'ammontare totale di denaro che corrisponde a ognuna di queste modalità di pagamento, occorre:

- Sapere quanto ogni *lender* ha pagato in ogni singolo prestito.
- Aggiungere per ogni prestito il suo *repayment_interval*.
- Per ogni *repayment_interval*, raggruppare per *lenders* e sommare.
- Aggiungere questo risultato a *lenders*, come richiesto.

### a. Sapere quanto ogni *lender* ha pagato in ogni prestito.

In [37]:
norm.head()

,loan_id,lenders,single_contribution,repayment_interval
0,483693,muc888,30.625,irregular
1,483693,sam4326,30.625,irregular
2,483693,camaran3922,30.625,irregular
3,483693,lachheb1865,30.625,irregular
4,483693,rebecca3499,30.625,irregular


### b. Aggiungere per ogni prestito il suo *repayment_interval*.

In [14]:
norm = norm.merge(loans[['loan_id', 'repayment_interval']], on='loan_id', how='left')

In [67]:
norm

,loan_id,lenders,single_contribution,repayment_interval
0,483693,muc888,30.625,irregular
1,483693,sam4326,30.625,irregular
2,483693,camaran3922,30.625,irregular
3,483693,lachheb1865,30.625,irregular
4,483693,rebecca3499,30.625,irregular
...,...,...,...,...
28293926,1206425,trogdorfamily7622,40.625,irregular
28293927,1206425,danny6470,40.625,irregular
28293928,1206425,don6118,40.625,irregular
28293929,1206486,alan5175,75.000,irregular


### c/d. Per ogni *repayment_interval*, raggruppare per *lender* e sommare; aggiungere questo risultato a *lenders*

In [68]:
possible_intervals = list(loans['repayment_interval'].unique())

for interval in possible_intervals:
    
    #Raggruppare per lender e sommare:
    money_by_interval = norm[norm['repayment_interval'] == interval].groupby(['lenders']).sum()
    
    #La chiave di money_by_interval è *lenders*. La si aggiunge anche come colonna.
    money_by_interval['permanent_name'] = money_by_interval.index
    
    #Aggiungere questo risultato a lenders.
    lenders = lenders.merge(money_by_interval[['permanent_name', 'single_contribution']], on='permanent_name', how='left')
    
    #Rinominare la colonna con il nome dell'intervallo.
    lenders[interval] = lenders['single_contribution']
    lenders = lenders.drop(['single_contribution'],axis = 1)
    
    #Porre i nan pari a 0.
    lenders[interval] = lenders[interval].fillna(0)

In [69]:
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,irregular,monthly,bullet,weekly
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0,46.839080,0.000000,0.0,0.0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0,32.471264,385.477101,0.0,0.0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0,0.000000,0.000000,0.0,0.0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0,0.000000,349.601088,0.0,0.0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0,0.000000,0.000000,0.0,0.0


# Esercizio 14

*Consegna*: What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?\
*Traduzione*: qual è l'occupazione con la media più alta di denaro prestato? La media va calcolata su tutti i *lenders* in cui l'occupazione è data.

**File necessari**: lenders, norm_with_contributions (dall'esercizio 8). 

### Selezione dei *lenders* di cui è data l'occupazione

In [84]:
lenders_occupation = lenders[lenders['occupation'].notnull()]
lenders_occupation.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,irregular,monthly,bullet,weekly
37,vikas1098,Vikas,Bengaluru,NaN,IN,1461301846,Software Engineer,NaN,2.0,NaN,0,62.5,29.166667,0.0,0.0
77,jack8790,Jack,Sydney,NSW,AU,1461303086,Business,NaN,NaN,NaN,0,0.0,0.000000,0.0,0.0
91,abpestcontrol,Above & Beyond Pest Control,Lake Worth,FL,US,1461300590,Owner,NaN,NaN,NaN,0,0.0,0.000000,0.0,0.0
98,oliver8929,Oliver,Utah,84057,US,1461301925,Nursing research,NaN,NaN,NaN,0,0.0,0.000000,0.0,0.0
177,markeylawfirm,Markey Stephen A III PC,Towson,MD,US,1461306342,Owner,NaN,NaN,NaN,0,0.0,0.000000,0.0,0.0


### Aggiunta dell'informazione sull'ammontare totale del denaro prestato da ogni *lender*

In [86]:
#Aggiunta dell'informazione
lenders_occupation = lenders_occupation[['permanent_name', 'occupation']].merge(norm_with_contributions,left_on = 'permanent_name',right_on = 'lenders', how='left')

#Pulizia del dataset
lenders_occupation = lenders_occupation.drop(labels='loan_id', axis=1)
lenders_occupation['single_contribution'] = lenders_occupation['single_contribution'].fillna(0)

lenders_occupation.head()

,permanent_name,occupation,single_contribution
0,vikas1098,Software Engineer,91.666667
1,jack8790,Business,NaN
2,abpestcontrol,Owner,NaN
3,oliver8929,Nursing research,NaN
4,markeylawfirm,Owner,NaN


### Individuazione del massimo

In [92]:
lenders_occupation[lenders_occupation['occupation'] == lenders_occupation.groupby('occupation').mean()['single_contribution'].idxmax()]

,permanent_name,occupation,single_contribution
52061,gooddogg1,www.linkedin.com/in/peacekeeper,8.642502e+06


# Esercizio 15
*Consegna*: Cluster the loans according to the year-month of disburse time.\
*Traduzione*: Raggruppa i prestiti a seconda di mese e anno di *disburse_time*

**File necessari**: loans

### Individuazione del mese e dell'anno di *disburse_time*

In [95]:
loans['month_year'] = loans['disburse_time'].dt.year.astype(str) + '-' + loans['disburse_time'].dt.month.astype(str)

### Raggruppamento dei prestiti

In [98]:
loans.groupby('month_year').count()['loan_id']

month_year
2005.0-4.0       203
2006.0-10.0      146
2006.0-11.0      744
2006.0-12.0      804
2006.0-3.0         1
               ...  
2017.0-9.0     17336
2018.0-1.0       339
2018.0-2.0       560
2018.0-3.0        65
nan-nan         2813
Name: loan_id, Length: 146, dtype: int64

# Esercizio 16

*Consegna*: For each country, compute its overall GNI, by multiplying the per capita GDP with its population.
*Traduzione*: per ogni paese, calcola il suo GNI totale, moltiplicato il GNI procapite per la sua popolazione.

**File necessari**: countries

In [100]:
countries['overall_gni'] = countries['gni']*countries['population']
countries.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,overall_gni
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,7.584413e+12
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,1.038975e+12
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,2.116096e+12
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,9.912214e+11
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh,5.502424e+11


# Esercizio 17

*Consegna*: Find the country with the highest rate of irregular repayment interval.\
*Traduzione*: trova il paese con il maggior rapporto di pagamenti con intervallo di ripagamento irregolare.

**File necessari**: loans

### Riscrittura della consegna.

Per ogni prestito è associato un paese e un intervallo di ripagamento. Quindi, a ogni paese corrisponderanno dei prestiti con intervallo di pagamento *monthly*, altri con intervallo di pagamento *weekly*, eccetera. Bisogna trovare il rapporto di ciascuno per paesi: cioè, per esempio, trovare che, di tutti i prestiti dati agli Stati Uniti, il 50% ha intervallo di pagamento *monthly*, il 25% dei prestiti ha intervallo *weekly*, eccetera. Poi, bisogna trovare il paese per cui è massima la percentuale di prestiti con intervallo di pagamento *irregular*.

### Controllo per gli eventuali valori nulli
Verifica che ogni prestito abbia il paese e l'intervallo di pagamento.

In [108]:
loans[loans['country_name'].isnull() | loans['repayment_interval'].isnull()]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,month_year


### Numero di prestiti per ogni paese e per ogni intervallo di pagamento.

In [157]:
loans_country_int = loans.groupby(['country_name', 'repayment_interval']).count()['loan_id']
loans_country_int.sort_index(inplace=True)
loans_country_int

country_name  repayment_interval
Afghanistan   bullet                   2
              irregular                1
              monthly               2334
Albania       bullet                  30
              irregular               32
                                    ... 
Zambia        irregular              144
              monthly                402
Zimbabwe      bullet                  11
              irregular             2283
              monthly               3219
Name: loan_id, Length: 238, dtype: int64

### Risistemazione della tabella

In [158]:
loans_country_int = loans_country_int.unstack()
loans_country_int = loans_country_int.fillna(0)
loans_country_int

repayment_interval,bullet,irregular,monthly,weekly
country_name,,,,
Afghanistan,2.0,1.0,2334.0,0.0
Albania,30.0,32.0,3013.0,0.0
Armenia,108.0,332.0,13512.0,0.0
Azerbaijan,5.0,833.0,9334.0,0.0
Belize,3.0,0.0,215.0,0.0
...,...,...,...,...
Vietnam,143.0,7949.0,13747.0,0.0
Virgin Islands,2.0,0.0,0.0,0.0
Yemen,0.0,616.0,3590.0,0.0


### Determinazione dei vari rapporti

In [159]:
loans_country_int['sum'] = loans_country_int['bullet'] + loans_country_int['irregular'] + loans_country_int['weekly'] + \
                            loans_country_int['monthly']

loans_country_int['irregular_rate'] = loans_country_int['irregular']/loans_country_int['sum']

### Individuazione del massimo

In [160]:
#La risposta è il paese che corrisponde a 'irregular_rate'
loans_country_int.idxmax()

repayment_interval
bullet               Colombia
irregular         Philippines
monthly                 Kenya
weekly                  Kenya
sum               Philippines
irregular_rate         Bhutan
dtype: object

# Esercizio 18

*Consegna*: Find the country with the highest fraction of loaned amount with irregular repayment interval.\
*Traduzione*: trova il paese con la massima frazione di denaro prestato con intervallo di ripagamento irregolare.

**File necessari**: loans

### Procedimento

Il procedimento è analogo all'esercizio precedente, ma questa volta bisogna fare la somma su *loan_amount*.  

### Somma totale di *loan_amount* per ogni paese e per ogni intervallo di pagamento.

In [161]:
amount_country_int = loans.groupby(['country_name', 'repayment_interval']).sum()['loan_amount']
amount_country_int.sort_index(inplace=True)
amount_country_int

country_name  repayment_interval
Afghanistan   bullet                  14000.0
              irregular                 350.0
              monthly               1953600.0
Albania       bullet                  44775.0
              irregular               43075.0
                                      ...    
Zambia        irregular              258875.0
              monthly                489675.0
Zimbabwe      bullet                   5500.0
              irregular             1714750.0
              monthly               4131625.0
Name: loan_amount, Length: 238, dtype: float64

### Risistemazione della tabella

In [162]:
amount_country_int = amount_country_int.unstack()
amount_country_int = amount_country_int.fillna(0)
amount_country_int

repayment_interval,bullet,irregular,monthly,weekly
country_name,,,,
Afghanistan,14000.0,350.0,1953600.0,0.0
Albania,44775.0,43075.0,4219500.0,0.0
Armenia,108800.0,749675.0,22092000.0,0.0
Azerbaijan,5000.0,2051800.0,12727825.0,0.0
Belize,1075.0,0.0,149100.0,0.0
...,...,...,...,...
Vietnam,343025.0,8960825.0,15377250.0,0.0
Virgin Islands,10000.0,0.0,0.0,0.0
Yemen,0.0,265525.0,3178475.0,0.0


### Determinazione dei rapporti

In [163]:
amount_country_int['sum'] = amount_country_int['bullet'] + amount_country_int['irregular'] + amount_country_int['weekly'] + \
                            amount_country_int['monthly']

amount_country_int['irregular_rate'] = amount_country_int['irregular']/amount_country_int['sum']

### Individuazione del massimo

In [164]:
amount_country_int.idxmax()

repayment_interval
bullet                  Kenya
irregular         Philippines
monthly              Cambodia
weekly                  Kenya
sum               Philippines
irregular_rate         Bhutan
dtype: object